In [7]:
import numpy as np
import pandas as pd
import re
import krippendorff
from collections import defaultdict, Counter
from itertools import combinations

In [3]:
df_mikkel = pd.read_json("annotations_mikkel.json")

#Fix of dataframe
def parse_label_string(label_str):
    return re.findall(r"'(.*?)'", label_str)

df_mikkel['original_label'] = df_mikkel['original_label'].apply(parse_label_string)

#Check:
df_mikkel["original_label"].iloc[0][0]

'None'

In [4]:
num_encoder = {'None': 0,
               'Minimal': 1,
               'Basic': 2,
               'Good': 3,
               'Excellent': 4}

N = len(df_mikkel)
score_data = []

for i in range(len(df_mikkel)):
    labels = [df_mikkel["our_label"].iloc[i]] + df_mikkel["original_label"].iloc[i]
    scores = [num_encoder[label] for label in labels]
    score_data.append(scores) #list of lists

In [ ]:
R = 5 #number of ratings

SD_list = []

for scores in score_data:
    c = len(scores)
    #The Squared Difference between unordered distinct pairs for one text:
    SD = (2 / (c * (c - 1))) * sum((
        (scores[j] - scores[k]) / (R - 1)) ** 2 
        for j in range(c) 
        for k in range(j+1,c))
    SD_list.append(SD)

MSD = np.mean(SD_list)
MSD


np.float64(0.08885416666666668)

In [13]:
reliability_data = np.array(score_data).T 
alpha = krippendorff.alpha(reliability_data=reliability_data, level_of_measurement='ordinal')
alpha

np.float64(0.382356809230209)

In [ ]:
D_o = MSD

# All annotation scores
all_scores = np.asarray(score_data).flatten()


label_counts = Counter(all_scores)  #dict
labels = list(range(R))
n_total = len(all_scores)  # total annotations

# Calculate D_e
De_numerator = 0
for i in range(R):
    for j in range(i + 1, R):
        a = labels[i]
        b = labels[j]
        n_a = label_counts[a]
        n_b = label_counts[b]
        delta = ((a - b) / (R-1)) ** 2
        De_numerator += n_a * n_b * delta

D_e =  De_numerator * 2 / (n_total * (n_total - 1))

alpha_simplifed = 1 - D_o / D_e
alpha_simplifed

np.float64(0.3718725190614425)

In [ ]:
#Testing for all distinct ordered pairs

R = 5 #number of ratings

SD_list = []

for scores in score_data[1:]:
    c = len(scores)
    SD = (1 / (c * (c - 1))) * sum(
        ((scores[j] - scores[k]) / (R - 1)) ** 2
        for j in range(c)
        for k in range(c)
        if j != k
    )
    SD_list.append(SD)

D_o = np.mean(SD_list)

# All annotation scores
all_scores = np.asarray(score_data).flatten()


label_counts = Counter(all_scores)  #dict
labels = list(range(R))
n_total = len(all_scores)  # total annotations

# Calculate D_e
De_numerator = 0
for i in range(R):
    for j in range(R):
        a = labels[i]
        b = labels[j]
        if a == b:
            continue
        n_a = label_counts[a]
        n_b = label_counts[b]
        delta = ((a - b) / (R-1)) ** 2
        De_numerator += n_a * n_b * delta

D_e =  De_numerator * 1 / (n_total * (n_total - 1))

alpha_ordered = 1 - D_o / D_e
alpha_ordered

np.float64(0.3718725190614425)